# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## symplectic generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1", "abAB", "cdCD", "efEF"]


["1", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


## Random Word generator

In [5]:
def word_generator(length: 1, genus: 3)
  rstr = ''
  length.times{|k| rstr += Rseq[0..(4*genus-1)].sample.to_s}
  return Word.new(rstr)
end
def wordgen(length=5)
  word_generator(length: length)
end;nil

# Goldman bracket & Turaev cobracket

## Term class and Expansion class

In [6]:
class Term < Hash
  '''
    A class express a term of W or wedge of W and W
    A Hash with keys :coeff, :words and :divs
  '''
  def initialize(coeff: 0, words: [], divs: [])
    begin      
      self[:coeff] = coeff
      self[:words] = words
      self[:divs] = divs
    rescue => e
      p e.backtrace
    end
  end
  
  def degree
    self[:words].size
  end
  
  def cyclic_reduce
    self.dup.tap {|t| t[:words].map!(&:cyclic_reduce)}
  end
  
  def wedge(a_Word)
    raise ArgumentError, a_Word.class unless a_Word.is_a?(Word)
    self.dup.tap {|t| t[:words] << a_Word}
  end
  
  def show
    coeff = (self[:coeff] == 1) ? '' : "(#{self[:coeff]})"
    body = self[:words].map(&:show).join("\u{2227}")
    coeff + body
  end
  
  def equiv?(a_term)
    '''
      return 1(match), -1(reversely match) and 0(not match).
    '''
    (self[:words].size).times do |k|
      flags = self[:words].zip(a_term[:words].rotate(k)).map{|pair| pair[0].is_cyclically_same?(pair[1])}
      if flags.all?{|tf| tf}
        return (-1)**k
        break
      end
    end
    return 0
  end
end; nil

In [7]:
class Expansion < Array
  '''
    An Array of Terms
  '''
  def ==(another)
    raise ArgumentError, another.class unless another.is_a?(self.class)
    diff = self.concat( another*(-1) ).simplify
    words = diff[0][:words]
    return (words.empty? || words.include?(Word.new))
  end
  
  def *(int)
    raise ArgumentError, int.class unless int.is_a?(Integer)
    return self.each_with_object(self.class.new) do |term, expn|
      term[:coeff] *= int
      expn << term
    end
  end
  
  def +(another)
    self.concat(another)
  end
  
  def wedge(a_Word)
    raise ArgumentError, a_Word.class unless a_Word.is_a?(Word)
    self.map!{|term| term.wedge(a_Word)} #.inject(self.class.new){|sum, term| sum << term}
  end
  
  def show(simplify_level=0)
    expn = case simplify_level
      when 1
        self.delete_if{|t| t[:coeff] == 0}
      when 2
        self.simplify
      else 
        self #self.rotate(0)
      end
    #---
    mstr = expn.map{|term| term.show}.join(' + ')
    return (mstr.empty?) ? '0' : mstr
  end

  def simplify
    expn = self.class.new
    monomials = self.map{|mono| mono.cyclic_reduce unless mono[:coeff] == 0}.compact
    until monomials.size == 0
      pop = monomials.pop
      pop[:divs] = [pop[:divs]]
      #---
      monomials.each do |mono|
        switch = mono.equiv?(pop)
        unless switch == 0
          pop[:coeff] += mono[:coeff] * switch
          pop[:divs] << mono[:divs]
          mono[:coeff] = nil
        end
      end
      expn << pop if pop[:coeff] != 0
      monomials.delete_if{|mono| mono[:coeff].nil?}
    end
    return (expn.empty?) ? (self.class.new << Term.new) : expn
  end
end; nil

### scratch

In [8]:
w = Word.new('abcde').cyclic_permutation
p w.class
nil

Word


In [9]:
terms = [
  Term.new,
  Term.new(words: [wordgen(8)], coeff: -1), 
  Term.new(words: [wordgen(3), wordgen(2)], coeff: 1)
]

terms.each do |t|
  p "#{t.show}, deg: #{t.degree}"
  printf ' |-reduce-> ' + t.cyclic_reduce.show + "\n"
end; nil

"(0), deg: 0"
 |-reduce-> (0)
"(-1)ddeedFCF, deg: 1"
 |-reduce-> (-1)ddeedFCF
"aEB∧ca, deg: 2"
 |-reduce-> aEB∧ca


In [10]:
count = 10**1 #**3 #
count.times do |k|
  expn = 5.times.map do |i|
    Term.new(divs: nil, 
      coeff: ( (-1)**(i.modulo(3)) - (-1)**((i+1).modulo(3)) )/2, 
      words: [3,3].map{|k| wordgen(k)}
      )
  end.inject(Expansion.new){|sum, term| sum << term}
  if expn.simplify.size < 4
    p expn.show(1) + " = " + expn.show(2)
    break
  end
end; nil

In [11]:
expn = 5.times.map do |i|
  Term.new(
    coeff: [-1,0,1].sample,
    words: [wordgen(rand(1..5))]
    )
end.inject(Expansion.new){|sum, term| sum << term}
expn = expn.wedge(Word.new('aB'))
p expn.show
nil

"(0)fc∧aB + (0)ddEb∧aB + (0)afCa∧aB + fcaA∧aB + (0)BceAe∧aB"


## Partition class

In [12]:
class Partition
  def initialize(word: Word.new, index: 0)
    @word = word
    s = word.flatten.size
    @index = index.modulo(s)
    @cutends = [
      {index: (index - 1).modulo(s), letter: @word[index-1], sign: -1}, 
      {index: index, letter: @word[index], sign: 1}
    ]
  end
  attr_reader :index, :cutends
  
  def show()
    "#{@cutends[0][:letter]}|#{@cutends[1][:letter]}"
  end
  def term()
    "#{@cutends[1][:letter]} - #{@cutends[0][:letter].inverse}"
  end
end; nil

### scratch

In [13]:
(-1).modulo(5)

4

In [14]:
w = Word.new('abcde')
p w.show
w.size.times do |k|
  ptn = Partition.new(word: w, index: k)
  printf ptn.show + ' , ' + ptn.term + "\n"
end; nil

"abcde"
e|a , a - E
a|b , b - A
b|c , c - B
c|d , d - C
d|e , e - D


## Linking of a pair of Pairtitions

In [15]:
def lk(ptn1, ptn2)
  '''
    Input: a pair of Partitions
    Output: -1 or 0 or 1
  '''
  [ptn1, ptn2].each{|ptn| raise ArgumentError, ptn.class unless ptn.is_a?(Partition)}
  ij_idxs = [0,1].product([0,1])
  total = ij_idxs.map{|i,j| epsilon( ptn1.cutends[i], ptn2.cutends[j] ) }.sum
  return total/2  #(-1)*(total/2)
end

#--------------------------------------------
def epsilon(h1, h2)
  '''
    Input: Hashes {index: a pos. Integer, letter: a Letter, sign: 1 or -1}
    Output: -1 or 1
  '''
  output = h1[:sign]*h2[:sign]
  letters = [h1, h2].map do |h|
    (h[:sign] == 1) ? h[:letter] : h[:letter].inverse
  end
  output *= unless (letters[0] == letters[1])
    (Rseq.index(letters[0]) < Rseq.index(letters[1])) ? 1 : -1
  else
    (h1[:index] > h2[:index]) ? h1[:sign]*(-1) : h2[:sign]
#    (h1[:index] < h2[:index]) ? h2[:sign] : h1[:sign]*(-1)
  end
  return output
end; nil

## Goldman bracket $\nabla$

In [16]:
def bracket(w1, w2)
  """
  retern: an Expansion (an Array of Terms {coeff: *, words: *, divs: *})
  """
  expn = Expansion.new
  unless (w1.show == '1' || w2.show == '1')
    ws = [w1, w2]
    (w1.size.times.to_a).product(w2.size.times.to_a).each do |inds|
      #--- the pair of partitions ---
      ptns = [0,1].map{|k| Partition.new(word: ws[k], index: inds[k])}
      #--- put the two words togather ---
      cpws = [0,1].map{|k| ws[k].cyclic_permutation(inds[k])}
      expn << Term.new(divs: inds, coeff: lk(*ptns), words: [cpws[0]*cpws[1]])
    end
  end
  return expn
end
#-----
def nabla(*args)
  '''
  For an expansion of degree 2, i.e. a formal sum of wedges.
  '''
  input = if args[0].is_a?(Expansion)
            args[0]
#           elsif args.size == 2 && args.all?{|w| w.is_a?(Word)}
#             Expansion.new << Term.new(coeff: 1, words: args)
          else
            msg = "args.size = #{args.size}, args[0].class = #{args[0].class}"
            raise ArgumentError, msg
          end
  output = input.map do |term|
    raise ArgumentError, term.inspect unless term.degree == 2
    2.times.map{|i| bracket(*(term[:words].rotate(i))) * (term[:coeff] * (-1)**i) }
  end.flatten.inject(Expansion.new){|sum, t| sum << t}
  return output
end; nil

### scratch

In [17]:
# w1 = Word.new(a[1], a[2], b[1].inverse)
# w2 = Word.new(b[2], a[1], a[1].inverse) #, b[2])
# w1, w2 = wordgen(2), wordgen(3)
w1, w2 = Word.new(a[1]), Word.new("aB")
printf [w1, w2].map(&:show).join(', ')  + "\n-------\n"

bra = bracket(w1, w2)

bra.each do |t|
  t.each{|k,v| printf "#{k}: " + ((k == :words) ? v[0].show : v.to_s) + ", "}
  printf "\n---\n"
end
puts bra.show, "= " + bra.show(2)
nil

a, aB
-------
coeff: 1, words: aaB, divs: [0, 0], 
---
coeff: 0, words: aBa, divs: [0, 1], 
---
aaB + (0)aBa
= aaB


In [18]:
t = Term.new(words: [wordgen(3), wordgen(2)], coeff: -1)
e = Expansion.new << t
p e.show
e2 = nabla(e)
p e2.show, e2.show(2), e2.wedge(Word.new('b')).show(2)
begin
  p nabla(t).show
rescue => exc
  p exc.inspect #class.to_s + ': ' + exc.message
end; nil

"(-1)bEb∧De"
"(0)bEbDe + (0)bEbeD + (0)EbbDe + (0)EbbeD + (-1)bbEDe + bbEeD + (0)DebEb + (0)DeEbb + (-1)DebbE + (0)eDbEb + (0)eDEbb + eDbbE"
"(2)Dbb + (-2)DebbE"
"(2)Dbb∧b + (-2)DebbE∧b"
"#<ArgumentError: args.size = 1, args[0].class = Term>"


## Turaev cobraket $\delta$

In [19]:
def cobracket(myw)
  """
  return: an Expansion (an Array of Terms {coeff: *, words: *, divs: *})
  """
  expn = Expansion.new
  unless myw.show == '1'
    ptn_num_pairs = (myw.size.times.to_a).combination(2)  #all the pair of partition numbers
    ptn_num_pairs.each do |i,j|
      #--- the pair of Partitions ---
      ptns = [i,j].map{|k| Partition.new(word: myw, index: k)}
      #--- divide the word ---
      words = myw.cyclic_permutation(i).split(j-i) #.reverse
      expn << Term.new(divs: [i,j], coeff: lk(*ptns), words: words)
    end
  end
  #-----
  return expn
end
#-----
def delta(*args)
  input = case args[0]
          when Expansion
            args[0]
          when Word
            Expansion.new << Term.new(coeff: 1, words: [args[0]])
          else
            msg = "args.size = #{args.size}, args[0].class = #{args[0].class}"
            raise ArgumentError, msg
          end
  output = input.map do |term|
    raise ArgumentError, term.inspect unless term.degree == 1
    cobracket(*term[:words]) * term[:coeff]
  end.inject{|sum, expn| sum + expn}
  return output
end; nil

### scratch

In [20]:
mw = Word.new('aabb') #wordgen(5) #Word.new(b[1], a[1], b[1]) #a1*Group.commutator(b[1], a[2]) #comms[1] #
printf mw.show + "\n-------\n"

cobra = cobracket(mw) #true) #

cobra.each do |t|
  t.each{|k,v| printf "#{k}: " + ((k == :words) ? v.map(&:show).join(' , ') : v.to_s) + ' / ' }
  printf "\n---\n"
end

puts cobra.show, "= " + cobra.show(2)
nil

aabb
-------
coeff: 1 / words: a , abb / divs: [0, 1] / 
---
coeff: 0 / words: aa , bb / divs: [0, 2] / 
---
coeff: 0 / words: aab , b / divs: [0, 3] / 
---
coeff: -1 / words: a , bba / divs: [1, 2] / 
---
coeff: -1 / words: ab , ba / divs: [1, 3] / 
---
coeff: 0 / words: b , baa / divs: [2, 3] / 
---
a∧abb + (0)aa∧bb + (0)aab∧b + (-1)a∧bba + (-1)ab∧ba + (0)b∧baa
= (-1)ab∧ba


# Well-definedness

In [21]:
def checker(count: 300, length: 8, &block)
  count.times do |counter|
    words, expn01, expn02 = *(block.call(length))
    #---
    if expn01 != expn02
      printf "\n words = #{words.map(&:show).join(' , ')} \n"
      printf "#{expn01.show(2)}  -  #{expn02.show(2)} != (0)"
      break
    else
      printf "O" if counter.modulo( (count>100) ? count/100 : 1) == 0
    end
    printf "\nPASS" if counter == count - 1
  end
end; nil

## bracket

In [22]:
checker(count: 100, length: 8) do |length|
  words = Array.new(2){|k| wordgen(rand(1..length))}
  expn01 = bracket(*words)
  expn02 = bracket(*(words.map(&:contract)))
  [words, expn01, expn02]
end; nil

OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
PASS

In [23]:
checker(count: 100, length: 6) do |length|
  words = Array.new(2){|k| wordgen(rand(1..length))}
  expn01 = bracket(*words)
  expn02 = bracket(*(words.map{|w| Group.conjugate(w, wordgen(2))}))
  [words, expn01, expn02]
end; nil

OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
PASS

## cobracket

In [24]:
checker(count: 2000, length: 6) do |length|
  word = wordgen(rand(1..length))
  expn01 = cobracket(word)
  expn02 = cobracket(word.contract)
  [[word], expn01, expn02]
end; nil

OOOOOOOOO
 words = ccEe 
(-2)c∧c + (-1)cc∧1  -  (-1)c∧c != (0)

In [25]:
checker(count: 500, length: 6) do |length|
  word = wordgen(rand(1..length))
  expn01 = cobracket(word)
  expn02 = cobracket(Group.conjugate(word, wordgen(3)))
  [[word], expn01, expn02]
end; nil

OOOOO
 words = EFEf 
(-2)E∧E  -  (-1)E∧E != (0)

# Conditions

## Jacobi Identity for $\nabla$

$$\nabla\circ(\nabla\otimes 1)\circ N = 0$$

In [26]:
max_length = 5

three_words = Array.new(3){|k| wordgen(rand(1..max_length))}
#--- display ---
puts three_words.map(&:show).join(" \u{2297} ") + "  |---> " + "\n------"
#---------------

total = 3.times.map do |k|
  triple = three_words.rotate(k)
  expn = bracket(*triple[0..1])
  #--- display ---
  puts "\t\u{2207}(#{triple[0..1].map(&:show).join(" \u{2297} ")}) \u{2297} #{triple[2].show} ="
  puts "\t\t+ (#{expn.show(1)}) \u{2297} #{triple[2].show}"
  #---------------
#  nabla( expn.tap{|e| e.each{|t| t[:words] << triple[2] }} )
  expn.map{|t| bracket(*(t[:words] << triple[2])) * t[:coeff] }.inject(Expansion.new){|sum, e| sum + e}
end.inject(Expansion.new){|sum, e| sum + e}

#--- display ---
printf "---------\n" + total.show(1) + "\n"
#---------------
"=" + total.show(2)

fEb ⊗ CCe ⊗ CaFcF  |---> 
------
	∇(fEb ⊗ CCe) ⊗ CaFcF =
		+ (EbfCCe + bfECCe + (-1)bfEeCC) ⊗ CaFcF
	∇(CCe ⊗ CaFcF) ⊗ fEb =
		+ (CCeFcFCa + CCeFCaFc + (-1)CeCCaFcF + CeCaFcFC + (-1)CeCcFCaF + CeCFCaFc + (-1)eCCCaFcF + eCCaFcFC + (-1)eCCcFCaF + eCCFCaFc) ⊗ fEb
	∇(CaFcF ⊗ fEb) ⊗ CCe =
		+ ((-1)CaFcFbfE + (-1)aFcFCfEb + aFcFCbfE + FcFCaEbf + (-1)cFCaFbfE + FCaFcEbf + FCaFcbfE) ⊗ CCe
---------
bfCCeECaFcF + (-1)bfCCeEaFcFC + bfCCeEcFCaF + (-1)bfCCeEFCaFc + fCCeEbaFcFC + (-1)CeEbfCCaFcF + CeEbfCaFcFC + (-1)CeEbfCcFCaF + CeEbfCFCaFc + (-1)eEbfCCCaFcF + eEbfCCaFcFC + (-1)eEbfCCcFCaF + eEbfCCFCaFc + bfECCeCaFcF + (-1)bfECCeaFcFC + bfECCeFcFCa + bfECCecFCaF + fECCebaFcFC + (-1)ECCebfFcFCa + (-1)ECCebfFCaFc + (-1)CebfECCaFcF + CebfECaFcFC + (-1)CebfECcFCaF + CebfECFCaFc + (-1)ebfECCCaFcF + ebfECCaFcFC + (-1)ebfECCcFCaF + ebfECCFCaFc + (-1)fEeCCbaFcFC + EeCCbfFcFCa + EeCCbfFCaFc + (-1)CCbfEeFcFCa + (-1)CCbfEeFCaFc + CbfEeCCaFcF + (-1)CbfEeCaFcFC + CbfEeCcFCaF + (-1)CbfEeCFCaFc + (-1)CCeFcFCabfE +

"=(0)"

In [27]:
checker(count: 5, length: 6) do |len|
  words = Array.new(3).map!{|w| wordgen(rand(1..len))}
  total = 3.times.map do |k|
    triple = three_words.rotate(k)
    bracket(*triple[0..1]).map{|t| bracket(*(t[:words] << triple[2])) * t[:coeff] }
#     nabla( bracket(*triple[0..1]).wedge(triple[2]) )
  end.flatten.inject(Expansion.new){|sum, e| sum << e}
#   p words.map(&:show), total.show(1), total.show(2)
  [words, total, Expansion.new]
end; nil

OOOOO
PASS

## Involutivity for $\nabla$ and $\delta$

$$\nabla\circ \delta = 0$$

In [28]:
len = 8
myw = wordgen(rand(1..len)) #Word.new('ffA') #

expn = cobracket(myw)
total = nabla( expn )
# total = cobracket(myw).map do |t|
#   bracket(*t[:words]) * t[:coeff]
# end.flatten.inject(Expansion.new){|sum,t| sum << t}

#--- display ---
puts "#{myw.show} |--\u{03B4}-->  " + expn.show(1)
puts "-----\n |--\u{2207}--> " + total.show(1)
#---------------
"=" + total.show(2)

Df |--δ-->  (-1)D∧f
-----
 |--∇--> 0


"=(0)"

In [29]:
checker(count: 100, length: 8) do |len|
  myw = wordgen(rand(1..len))
  total = nabla(cobracket(myw))
  [[myw], total, Expansion.new]
end; nil

OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
PASS

## Compatibility condition for $\nabla$ and $\delta$

$\forall v,w \in \pi$,
$$
\delta([v,w]) = v\cdot\delta(w)- w\cdot\delta(v),
$$
where 
$$
u\cdot(x\otimes y) := [u,x]\otimes y + x\otimes[u,y].
$$

**Note**: From the last formula, we immediately obtain the following:
$$
u \cdot(x\wedge y) = [u,x]\wedge y - [u,y]\wedge x.
$$
Futhermore, we may define $(x\wedge y)\cdot u$ as follows:
$$
(x\wedge y)\cdot u := x\wedge [u,y] - y \wedge [u,x],
$$
and have
$$
(x\wedge y)\cdot u = u \cdot(x\wedge y). 
$$

In [30]:
v, w = wordgen(3), wordgen(5) #samples[:s2][0] #Word.new(a[1], a[2]),samples[:s4][0] #Word.new(b[1])
#--- display ---
puts "#{v.show} \u{2297} #{w.show} \n====="
#---------------

#--- Left-hand eq. -------------------
bra = bracket(v, w)
lhe = delta( bra )
#--- display ---
puts "[#{v.show}, #{w.show}] = " + bra.show(1) + "\n---"
printf "Left-hand Eq.:  " + lhe.show(2) + "\n====="
#---------------

#--- Right-hand eq. -------------------
double_idx = [0,1].product([0,1])
rhe = double_idx.map do |i,j|
  pair_i, sign_i = [v,w].rotate(i), (-1)**i
  u = pair_i[0]
  cobra = cobracket(pair_i[1]) * sign_i #; printf "#{u}.\u{03B4}(#{pair_i[1].show}) = #{u}.(#{cobra.show(1)})\n---\n"
  #---
  cobra.map do |term|
    pair_j, sign_j = term[:words].rotate(j), (-1)**j
    #---
    x, y = *pair_j
    bra = bracket(u, x) * (term[:coeff] * sign_j) #; printf "\t[#{u}, #{x}] \u{2297} #{y} \n" + "\t=(#{bra.show(1)}) \u{2297} #{y}"
    bra.wedge(y)
  end.inject{|total, expn| total + expn }
#   printf "\n---\n"
end.inject{|total, expn| total + expn }
#-----
printf "\nRight-hand Eq.:  "+ rhe.show(2)

#-----------------------
lhe == rhe

bdF ⊗ BFdeB 
=====
[bdF, BFdeB] = bdFFdeBB + bdFdeBBF + (-1)bdFBBFde + dFbFdeBB + dFbeBBFd + (-1)dFbBBFde + (-1)FbdFdeBB + (-1)FbddeBBF + (-1)FbdeBBFd
---
Left-hand Eq.:  B∧FdFbdeB + (-1)BBF∧dFbde + de∧BFdF + (-1)b∧deBBFdF + (2)FbdeBBF∧d + B∧FFbddeB + (-1)b∧ddeBBFF + (-1)FbddeBB∧F + Fbdde∧BBF + de∧BBFbdF + d∧FdeBBFb + (-1)F∧dFdeB + (-1)d∧edFBF + F∧dedFB + (-1)d∧FBFde + (-1)F∧ddFbeBB + (-1)B∧FddFbeB + BBF∧ddFbe + b∧eBBFddF + (-1)de∧BBdFbF + d∧eBBdFbF + b∧FdeBBdF + dFbFdeB∧B + (-1)d∧ebdFBBF + F∧debdFBB + B∧FdebdFB + (-1)d∧FBBFdeb + (-1)b∧dFBBFde + (-1)de∧BdFF + d∧eBdFF + d∧FFdeB
=====
Right-hand Eq.:  BBFdebF∧d + BFdeF∧d + (2)deBBFFb∧d + (-1)FdeBF∧d + (-1)FdeBBFb∧d + BBFdedF∧b + eBBFdFd∧b + (-1)eBBFddF∧b + deBBFFd∧b + (-1)FdeBBdF∧b + (-1)FbdFBB∧de + dFbFBB∧de + (-1)dFBF∧de + dFFB∧de + (-1)dFbFeBB∧d + dFBFe∧d + (-1)dFFeB∧d + bdFBBFe∧d + (-1)FbddeBB∧F + (-1)dFBde∧F + dFbeBBd∧F + (-1)bdFBBde∧F + dFdeB∧F + Fbdde∧BBF + (-1)dFbed∧BBF + dFbde∧BBF + (-1)FbdeBFd∧B + (-1)FbddeBF∧B + dFbeBFd∧B + dF

true

An example whose results of the both side of the compatibility condition are very long.

In [31]:
v,w = Word.new('fbC'), Word.new('CfDBe')
nil

# Experiments

## $\delta(xy)$

In [32]:
level = 1
ws = [3,1].map{|k| wordgen(k) }

expn01 = cobracket(ws[0]*ws[1])
expn02 = bracket(*ws)
expn03 = delta(expn02)
expn04 = expn01.concat(expn03*(-1))

puts "x , y = " + ws.map(&:show).join(' , ') + "\n------\n"
puts ws.map{|w| "\u{03b4}(#{w.show}) = " + cobracket(w).show(level) }.join(' , ') + "\n----"

puts "\u{03b4}(x*y) = " + expn01.show(level) + "\n---"
puts "\u{2207}(x,y) = " + expn02.show(level)
puts "\u{03b4}(\u{2207}(x,y)) = " + expn03.show(level)
# puts "\u{03b4}(x*y)-\u{03b4}(\u{2207}(x,y)) = " + expn04.show(level)
nil

x , y = fDB , f
------

δ(fDB) = fD∧B + (-1)D∧Bf , δ(f) = 0
----
δ(x*y) = f∧DBf + (-1)D∧Bff + (-1)B∧ffD
---
∇(x,y) = 0
δ(∇(x,y)) = 0


## Cobracket calclation samples

In [24]:
a1, b1 = Word.new(a[1]), Word.new(b[1])
c = Group.conjugate(b1, a1)

samples = {
  s0: [
    Word.new,
    a1, 
    a1*Word.new(b[2]),
    a1*(Word.new(b[2]).inverse),
    a1*c*(c.inverse),
  ],
  s1: [
    a1^2,
    a1^3,
    a1^4,
    c,
    comms[1],
    Word.new(b[1])*a1*Word.new(b[1])
  ],
  s2: [
    a1*(comms[2].inverse),
    a1*(c.inverse)*(comms[2].inverse)*c,
    a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse),
    (a1*(comms[2].inverse)*Group.commutator(comms[2], c.inverse)).cyclic_reduce
  ],
  s3: [
    a1*comms[2],
    a1*(comms[2]^2),
    a1*comms[2]*c*comms[2]*(c.inverse),
    a1*(comms[2]^2)*Group.commutator(comms[2].inverse,c)
  ],
  s4: [
    a1*(Word.new(a[2])^3),
    a1*(Word.new(a[2], b[2].inverse)^3)
  ],
  s5: [
    a1*Group.commutator(a[2], a[3]),
    a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse)
  ],
  s6: [
    a1*Group.commutator(b[1], a[2]),
    a1*(a1.inverse)*Word.new(b[1])*a1,
    c*(a1.inverse)*a1,
    c*(c.inverse),
    Word.new(a[1], b[1], b[1].inverse, b[2])
  ],
  s7: [
    a1*(Group.conjugate(a1, b1.inverse))*a1*(b1.inverse),
    a1*b1*((a1*(b1.inverse))^2),
    a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse),
    (a1*Group.conjugate(b1.inverse, a1)*a1*(b1.inverse)).contract,
    a1*a1*(Word.new(b[2].inverse)^2)
    ]
  }

arr = samples #.slice(:s7) #:s0, :s6) #:s2) #, :s3) #:s0, :s1) #
arr.each do |k, v|
  printf "--- #{k} ---\n "
  v.each{|s| puts s.show + "  |--\u{03B4}-->  " + cobracket(s).show(2) }
end;nil

--- s0 ---
 1  |--δ-->  (0)
a  |--δ-->  (0)
ad  |--δ-->  (-1)a∧d
aD  |--δ-->  (0)
aabAaBA  |--δ-->  (-1)a∧1
--- s1 ---
 aa  |--δ-->  a∧a
aaa  |--δ-->  (0)
aaaa  |--δ-->  aa∧aa
abA  |--δ-->  (-1)b∧1
abAB  |--δ-->  (0)
bab  |--δ-->  (0)
--- s2 ---
 adcDC  |--δ-->  (-1)a∧dcDC
aaBAdcDCabA  |--δ-->  (-1)aBAdcDCab∧1
adcDCcdCDaBAdcDCabA  |--δ-->  (-2)aBAdcDCab∧1
aBAdcDCab  |--δ-->  (0)
--- s3 ---
 acdCD  |--δ-->  (0)
acdCDcdCD  |--δ-->  acdCD∧cdCD
acdCDabAcdCDaBA  |--δ-->  (0)
acdCDcdCDdcDCabAcdCDaBA  |--δ-->  (0)
--- s4 ---
 accc  |--δ-->  acc∧c + ac∧cc
acDcDcD  |--δ-->  acDcD∧cD + acD∧cDcD
--- s5 ---
 aceCE  |--δ-->  (-1)aceC∧E + ace∧CE + (-1)ac∧C
acdCDefEFceCEfeFEdcDC  |--δ-->  E∧feFEdcDCacdCDefEFceC + (-1)CE∧feFEdcDCacdCDefEFce + C∧feFEdcDCacdCDefEFc
--- s6 ---
 abcBC  |--δ-->  cB∧Cab + (-1)abcB∧C + abc∧BC
aAba  |--δ-->  (0)
abAAa  |--δ-->  (-1)b∧1
abAaBA  |--δ-->  (0)
abBd  |--δ-->  1∧da + (-1)a∧d
--- s7 ---
 aBabaB  |--δ-->  (0)
abaBaB  |--δ-->  (0)
aaBAaB  |--δ-->  aB∧aB
aaBB  |--δ--> 

## The cobracket completely detects simpleness

In [25]:
word = Word.new('aabb') #wordgen(rand(1..4)) #Word.new('cEEb') #
puts "#{word.show} |--> #{cobracket(word).show(2)}"

power = word^3
puts "#{power.show} |--> #{cobracket(power).show(2)}"
nil

aabb |--> (-1)ab∧ba
aabbaabbaabb |--> (-3)ab∧baabbaabba + (3)ba∧abbaabbaab + (-3)abbaab∧baabba


## Degree-3 simpleness of $a_{1}[a_{2}, a_{3}]$

In [26]:
x = a1*(Group.commutator(a[2], b[2])^2)
printf x.show + " |-\u{03b4}-> " + delta(x).show(2) + "\n"

count = 100
count.times do |counter|
  three_words = 3.times.map{|k| word_generator(length: rand(1..1), genus: 3)}
  gamma = three_words.inject{|memo, w| Group.commutator(memo, w)}
  cobra = cobracket(x*gamma)
  #--- Progress bar & result ---
  if cobra.show(2) == '(0)'
    puts "\n"
    p gamma.show, gamma.contract.show 
    break
  else
    printf "O" #if counter.modulo(count/10) == 0
  end
  #-----------------------------
end; nil

acdCDcdCD |-δ-> acdCD∧cdCD
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

In [36]:
w = a1*Group.commutator(Group.commutator(a1, b1.inverse), a1)
p w.show

cobracket(w).show(2)

"aaBAbaBabAA"


"(0)"

# TODO

* [ ] Expansion#wedge が #tensor と混同されている状況を解消する。<-- 解消してることを確認。
* [ ] wedge の一方に 1 があるものの扱いをはっきりさせる。<-- ゼロと認識させる。
* [ ] $x\wedge x$ をゼロと認識させる。
---
* [ ] cobracket を再帰的に作用させて、完全に分解することに何らかの意味があるか?
* [ ] $\ell_{2}$ の計算と cobracket との関係は? とくに両者にある partition について。
---
* [x] Expansion#show のオプションに "係数 0 の項だけ消去" を加えたい。
* [x] Compatibility が全然ダメ。
* [x] aaa などの Word について、動作が少し変。
* [x] Extension class のメソッドとして、係数 0 を消去したり、共通項で整理したりできるようにしたい。